<a href="https://colab.research.google.com/github/Yunpei24/BigDataBase/blob/main/Activite2_Question4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ACTIVITE2

Installation du JDK

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Création de la variable d'environnement <JAVA_HOME> pour situer l'emplacement d'installationde Java 

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Installation du framework Hadoop

Téléchargement depuis les archives de la fondation Apache

In [ ]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

Extraction de l'archive

In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Copie du dossier extrait dans l'emplacement <user/local>

In [5]:
!cp -r hadoop-3.3.0/ /usr/local/

## Programmation de tâches distribuées avec <i>MapReduce</i>

Création d'un repertoire <myinput> pour contenir le jeu de données à tester durant cet exercice e d'un second pour les résultats du traitement distribué

In [6]:
!mkdir -p ~/myinput
!mkdir -p ~/myoutput

Télachargement du jeu de données dans le fichier <u>purchases.txt</u>

In [7]:
!curl -L -o 'purchases.txt' 'https://drive.google.com/u/0/uc?id=1NS-PSXW8bSNpzFH4XRbtmMnMGhXBdYy6&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  201M  100  201M    0     0  45.9M      0  0:00:04  0:00:04 --:--:-- 55.6M


Déplacement du fichier <u>purchases.txt</u> dans le répertoire <myinput>

In [8]:
!mv purchases.txt ~/myinput/purchases.txt

Vérification que les fichiers ont été bien copiés

In [9]:
!ls /usr/myinput

ls: cannot access '/usr/myinput': No such file or directory


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Affichage des premiers lignes du fichier. Le format des enregistrement est le suivant:
<table border='1'><tr>
<td>Date</td><td>Heure</td><td>Magasin</td><td>Produit</td><td>Montant</td><td>Moyen_de_paiement</td>
</tr></table>
La tabulation <b>\t</b> est utilisée comme séparateur de colonne ✅

In [ ]:
!head -50  ~/myinput/purchases.txt

#QUESTION4 : Le nombre total des ventes et la valeur totale des ventes de tous magasins confondus

In [12]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(store, "\t1\t", amount)

In [28]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(item, "\t", amount)

In [29]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper.py', 'w') as f:
  f.write(In[28]) 
f.close()

In [19]:
!chmod u+rwx /content/mapper.py

In [38]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py

Men's Clothing 	 214.05
Women's Clothing 	 153.57
Music 	 66.08
Pet Supplies 	 493.51
Children's Clothing 	 235.63
Men's Clothing 	 247.18
Cameras 	 379.6
Consumer Electronics 	 296.8
Toys 	 25.38
Toys 	 213.88


In [30]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotal = 0
compteur = 0

for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  compteur +=1  
  salesTotal += float (data[1])

if compteur != 0:
  print("Nombre total de vente = ", compteur, "Montant total = ", str(salesTotal))

In [16]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotalVente = 0
salesTotalAmount = 0
oldKey = None

print("Magazins\tNbresVentes\tChiffreAffaire")
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 3:
    continue
  
  thisKey, thisVente, thisAmount = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotalVente), "\t", str(salesTotalAmount))
    salesTotalVente = 0
    
  oldKey = thisKey
  salesTotalVente += int (thisVente)
  salesTotalAmount += float(thisAmount) 

if oldKey != None:
  print(oldKey, "\t", str(salesTotalVente), "\t", str(salesTotalAmount))

Magazins	NbresVentes	ChiffreAffaire


In [31]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer.py', 'w') as f:
  f.write(In[30]) 
f.close()

In [32]:
!chmod u+rwx /content/reducer.py

In [33]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py | sort | python3 /content/reducer.py

Nombre total de vente =  10 Montant total =  2325.6800000000003


In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

In [25]:
!ls ~/myoutput

part-00000  _SUCCESS


In [37]:
!tail -n 30 ~/myoutput/part-00000

Nombre total de vente =  4138476 Montant total =  1034457953.2599641	
